**Parameterizing small molecule for gromacs/Amber simulations**

first draw your molecule, and minimize it with MM level. Now prepare a .com file using antechamber via the following command

In [ ]:
antechamber -i name.pdb -fi pdb -o *.com -fo gcrt

### this will print .com file with title #HF/6-31G* SCF=tight Test Pop=MK iop(6/33=2) iop(6/42=6). Now you need to run guassian program in order to get .log file, which will be used in the next step

In [ ]:
antechamber -i *.log -fi gout -o *.mol2 -fo mol2 -c resp


*#you can calculate resp charges by Using Multiwfn (http://sobereva.com/multiwfn) to calculate RESP charges. The procedure is much easier and the functionality is more powerful than Antechamber, and you do not need to boring with connectivity. Please follow the examples in Section 4.7.7 of Multiwfn manual.*

In [ ]:
$tleap
Source leaprc.protein.ff14SB
Source leaprc.water.opc 
Source leaprc.protein.ff14SB 
Source leaprc.water.opc
Source leaprc.gaff2
ligand = loadmol2 genistein_final.mol2 
check ligand
loadamberparams genistein_final.mfrcmod
saveoff ligand ligand.lib 
setbox ligand vdw 6
#solvatebox ligand TIP3PBOX 10.0
saveamberparm ligand ligand.inpcrd ligand.prmptop
quit


you will get final ligand in water output with the paramters compatible for amber simulation. Note that solvatebox is additional command if you want to add water molecule, however if you just interested ligand parameterization and interested in gromacs topology, you can skip this step.

In case you want to visualize the output file in .pdb format, you can use following command

In [ ]:
ambpdb -p ligand.inpcrd -c ligand.prmptop > ligand.pdb

gromacs to amber topology can be converted via python module called parmed. make a script.py file via the following input

In [ ]:
import parmed as pmd
parm = pmd.load_file('ligand.prmptop','ligand.inpcrd')
parm.save('ligand.itp',format='gromacs')
parm.save('ligand.gro')
parm.save('ligand.top') #,format='gromacs')

In [ ]:
python script.py

You will get gromacs input files and you are good to go with gromacs simulations. In case if want to simulate ligand in water, i am here giving you .mdp files (em.mdp, eq.mdp, run.mdp), sample .top file (system_EMPTY.top), and you just need to execute bash script (prepare_1mol_AA_system.sh), it will simply take care of everything.

However, if you want to simulate using sander, you can use following command

In [ ]:
sander -O -i 01_Min.in -o 01_Min.out -p ligand.pmptop -c ligand.inpcrd -r 01_Min.ncrst -inf 01_Min.mdinfo